In [ ]:
from final_evaluation import *

In [ ]:
import os
import pandas as pd

test_dir = '/media/minhduck/One Touch/official/images/test_retrieval'
output_dir = os.path.join('working/', test_dir.replace('/', '_'))

In [ ]:
print("Capturing depth buffer of stl meshes...")
capture_depth_buffer(test_dir, output_dir)

In [ ]:
from argparse import Namespace

# Model config
args = Namespace(
  model_name='caformer_s36.sail_in22k_ft_in1k_384',
  no_head=True,
  device='cuda',
  save_path='./working/trained_models/oml/ckpt_05',
)


      
      
image_retrieval(args, test_dir, output_dir)

In [ ]:
pd.read_csv(os.path.join(output_dir, 'pred.csv'), skiprows=1, names=['query_name', 'correct_output'])

In [ ]:


def calculate_mrr_at_5(gt_csv, pred_csv):
    # gt_df = pd.read_csv(gt_csv, header=None, names=['query_name', 'correct_output'])
    # pred_df = pd.read_csv(pred_csv, header=None, names=['query_name', 'predictions'])
    gt_df = pd.read_csv(gt_csv, skiprows=1, names=['query_name', 'correct_output'])
    pred_df = pd.read_csv(pred_csv, skiprows=1, names=['query_name', 'predictions'])
    
    reciprocal_ranks = []

    for _, gt_row in gt_df.iterrows():
        query_name = gt_row['query_name']
        correct_output = gt_row['correct_output'].split(',')[0]
        
        pred_row = pred_df[pred_df['query_name'] == query_name]
        if pred_row.empty:
            reciprocal_rank = 0
        
        predicted_outputs = pred_row.iloc[0]['predictions'].split(',')
        print(predicted_outputs, gt_row)
        try:
            rank = predicted_outputs[:5].index(correct_output) + 1
            reciprocal_rank = 1 / rank
        except ValueError:
            reciprocal_rank = 0
        print("=>", reciprocal_rank)
        
        reciprocal_ranks.append(reciprocal_rank)
    
    # Calculate Mean Reciprocal Rank (MRR)@5
    mrr_at_5 = sum(reciprocal_ranks) / len(reciprocal_ranks)
    
    return mrr_at_5

calculate_mrr_at_5(os.path.join(test_dir, 'labels.csv'), os.path.join(output_dir, 'pred.csv'))